In [89]:
import numpy as np
import matplotlib.pyplot as plt
import math
import functools
import scipy

In [2]:
def same(*args):
    for arg in args:
        assert args[0] == arg
    return args[0]

# Probleme

## 1

In [88]:
def trapezoid_method(a, b, f, n):
    h = (b-a)/n
    res = (
        (b - a) / (2*n)
        * (
            f(a) + f(b)
            + 2*sum(
                f(a + k*h)
                for k in range(1, n)
            )
        )
    )
    return res
trapezoid_method(0, math.pi/2, math.cos, 20)

0.999485905248533

In [80]:
def rectangle_method(a, b, f, n):
    h = (b-a)/n
    res = (
        (b - a) / n
        * sum(
            (f(a + (k-1)*h) + f(a + k*h)) / 2
            for k in range(1, n+1)
        )
    )
    return res
rectangle_method(0, math.pi/2, math.cos, 20)

0.999485905248533

In [81]:
def simpson_method(a, b, f, n):
    assert n % 2 == 0
    m = n//2
    h = (b-a)/n
    res = (
        (b - a) / (6*m)
        * (
            f(a) + f(b)
            + 2*sum(
                f(a + 2*k*h)
                for k in range(1, m)
            )
            + 4*sum(
                f(a + (2*k-1)*h)
                for k in range(1, m+1)
            )
        )
    )
    return res
simpson_method(0, math.pi/2, math.cos, 20)

1.0000002115465914

## 3

In [61]:
def adaptive_quad(a, b, f, met, tol, *, m):
    if abs(met(a, b, f, m) - met(a, b, f, 2*m)) < tol:
        return met(a, b, f, 2*m)
    else:
        return (
              adaptive_quad(a, (a+b)/2, f, met, tol, m=m)
            + adaptive_quad((a+b)/2, b, f, met, tol, m=m)
        )

In [82]:
adaptive_quad(0, math.pi/2, math.cos, trapezoid_method, 0.0001, m=4)

0.999938335910324

In [83]:
adaptive_quad(0, math.pi/2, math.cos, rectangle_method, 0.0001, m=4)

0.9999383359103238

In [84]:
adaptive_quad(0, math.pi/2, math.cos, simpson_method, 0.0001, m=4)

1.0000005166847064

## 4

In [85]:
def romberg_method(a, b, f, n):
    def h(k: int):
        return (b - a) / (2**(k-1))
    
    @functools.cache
    def R(i: int, j: int):
        if i == 1 and j == 1:
            return h(1)/2 * (f(a) + f(b))
        elif j == 1:
            return 1/2 * (
                R(i-1, 1)
                + h(i-1)*sum(
                    f(a + (k - 1/2)*h(i-1))
                    for k in range(1, 2**(i-2)+1)
                )
            )
        else:
            return (
                (4**(j-1)*R(i, j-1) - R(i-1, j-1))
                / (4**(j-1) - 1)
            )

    return R(n, n)

romberg_method(0, math.pi/2, math.cos, 20)

1.0000000000000027

## 5

In [78]:
def quadstep(f, a, b, eps, fa, fc, fb):
    h = b-a
    c = (a+b)/2
    fd = f((a+c)/2)
    fe = f((c+b)/2)
    Q1 = h/6 * (fa + 4*fc + fb)
    Q2 = h/12 * (fa + 4*fb + 2*fc + 4*fe + fb)
    if abs(Q1 - Q2) < eps:
        Q = Q2 + (Q2 - Q1) / 15
    else:
        Qa = quadstep(f, a, c, eps, fa, fd, fc)
        Qb = quadstep(f, c, b, eps, fc, fe, fb)
        Q = Qa + Qb
    return Q
    

def adaquad(f, a, b, eps):
    c = (a+b)/2
    fa = f(a)
    fb = f(b)
    fc = f(c)
    return quadstep(f, a, b, eps, fa, fc, fb)

adaquad(math.sin, 0, math.pi, 0.001)

1.9999701192848807

# Probleme practice

## 2

In [92]:
def f(x):
    return math.e**(x**2)

a = 1
b = 2
n = 10

print(scipy.integrate.quad(f, a, b)[0])

print(trapezoid_method(a, b, f, n))
print(rectangle_method(a, b, f, n))
print(simpson_method(a, b, f, n))

15.166784148626101
15.166784148626101
14.99252700644484
14.989976019600046
